In [237]:
import pandas as pd

file_mcu = 'C:/Users/serha/PycharmProjects/Temp/BatterySolutions/Validation_Dataset/Neocity_Parse_Dosyaları_18_Şubat_19_Mart_2025/mergedparse_Arac200727_MCU_18Subat_19Mart.csv'
file_dcdc = 'C:/Users/serha/PycharmProjects/Temp/BatterySolutions/Validation_Dataset/Neocity_Parse_Dosyaları_18_Şubat_19_Mart_2025/mergedparse_Arac200728_DCDC_18Subat_19Mart.csv'

In [238]:
# Read a few lines manually to inspect the delimiter and structure
with open(file_mcu, 'r', encoding='utf-8') as f:
    mcu_lines = [next(f) for _ in range(10)]

with open(file_dcdc, 'r', encoding='utf-8') as f:
    dcdc_lines = [next(f) for _ in range(10)]

mcu_lines, dcdc_lines

(['Date Time;2_0440_RollingCounter;2_0440_HighPowerVoltageStatus;2_0440_HighPowerVoltage;2_0440_HighPowerCurrentStatus;2_0440_HighPowerCurrent;2_0440_AuxiliaryVoltageStatus;2_0440_AuxiliaryVoltage;2_0441_RollingCounter;2_0441_MotorTorqueEstimatedStatus;2_0441_RegenTorqueAvailStatus;2_0441_TractionTorqueAvailStatus;2_0441_RegenTorqueAvail;2_0441_TractionTorqueAvail;2_0441_MotorTorqueEstimated;2_0442_FaultPhaseToChassisStatus;2_0442_FaultInterlockMotorCableStatus;2_0442_FaultInterlockBatteryCableStatus;2_0442_FaultDriveCoverWasOpenedStatus;2_0442_FaultDriveCoverWasOpened;2_0442_FaultInterlockMotorCable;2_0442_FaultInterlockBatteryCable;2_0442_FaultPhaseToChassis;2_0443_RollingCounter;2_0443_EventId;2_0444_MajorMcuProtocol;2_0444_MinorMcuProtocol;2_0445_VmuRequestId;2_0446_BootNumber;2_0447_CommandModeMinVoltage;2_0447_CommandModeMaxVoltage;2_0447_McuRequestId;2_0448_TotalMotorRotation;2_0449_RollingCounter;2_0449_MotorCoolingFlowRequestStatus;2_0449_MotorCoolingFlowRequest;2_0449_DriveCo

In [239]:
# Load the files again using correct delimiter and decimal symbol
df_mcu = pd.read_csv(file_mcu, delimiter=';', decimal=',', encoding='utf-8', low_memory=False)
df_dcdc = pd.read_csv(file_dcdc, delimiter=';', decimal=',', encoding='utf-8', low_memory=False)

(df_mcu.shape, df_dcdc.shape)

((46754, 71), (42497, 114))

In [240]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Select features and target for training
features = ["1_F231_Avarage_Battery_Voltage", "1_F231_Current_Charge_Discharge", "1_F230_SOC", "1_F232_High_temp", "1_F232_Low_temp", "1_F230_The_Remaining_Capacity"]
target = "1_F230_The_Remaining_Capacity"

# Drop missing values and convert everything to float
df_model = df_dcdc[features + [target]].dropna().astype("float32")

X = df_model[features]
y = df_model[target]

# Ensure y is a Series
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🌲 Validation MSE: {mse:.2f}")
print(f"🌲 Validation R² Score: {r2:.4f}")


🌲 Validation MSE: 65.24
🌲 Validation R² Score: 0.9869


In [241]:
y_pred

array([ 63.,  68.,  83., ..., 247., 135., 121.])

In [242]:
X_test

,1_F231_Avarage_Battery_Voltage,1_F231_Current_Charge_Discharge,1_F230_SOC,1_F232_High_temp,1_F232_Low_temp,1_F230_The_Remaining_Capacity,1_F230_The_Remaining_Capacity
23288,599.799988,7.500000,23.0,16.0,11.0,63.0,63.0
22906,603.099976,8.000000,25.0,14.0,11.0,68.0,68.0
12482,612.799988,-104.599998,29.0,25.0,17.0,83.0,83.0
41591,670.200012,8.100000,87.0,23.0,19.0,245.0,245.0
31313,657.299988,10.700000,84.0,28.0,26.0,238.0,238.0
...,...,...,...,...,...,...,...
23588,686.400024,12.200000,97.0,27.0,23.0,274.0,274.0
2266,624.400024,-148.699997,53.0,26.0,22.0,150.0,150.0
36778,665.200012,37.500000,87.0,39.0,34.0,247.0,247.0
18143,617.599976,6.600000,47.0,27.0,23.0,135.0,135.0


In [250]:
file_dcdc_2 = 'C:/Users/serha/PycharmProjects/Temp/BatterySolutions/Validation_Dataset/Neocity_Parse_Dosyaları_19_Ocak_19_Şubat_2025/mergedParse_Araç200728_DCDC_20 Ocak - 19 Subat 2025.csv'

In [251]:
df_dcdc_validation = pd.read_csv(file_dcdc_2, delimiter=';', decimal=',', encoding='utf-8', low_memory=False)

In [252]:
# Select features and target for training
features = ["1_F231_Avarage_Battery_Voltage", "1_F231_Current_Charge_Discharge", "1_F230_SOC", "1_F232_High_temp", "1_F232_Low_temp", "1_F230_The_Remaining_Capacity"]
target = "1_F230_The_Remaining_Capacity"

In [253]:
# Drop missing values and convert everything to float
df_model = df_dcdc_validation[features + [target]].dropna().astype("float32")

X = df_model[features]
y = df_model[target]

# Ensure y is a Series
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [254]:
# Predict and evaluate
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🌲 Validation MSE: {mse:.2f}")
print(f"🌲 Validation R² Score: {r2:.4f}")

🌲 Validation MSE: 742.00
🌲 Validation R² Score: 0.9043
